In [1]:
import matplotlib.pyplot as plt
import hyperspy.api as hs
import numpy as np
import exspy
import scipy
import os
import mplcyberpunk
import matplotlib 
import h5py
import hdf5plugin
from py4Dutils.Added_Functions_For_EELS_Mapping import Set_Wrong_Pixels_to_NaN, Set_Wrong_Pixels_to_Zero, Set_Wrong_Pixels_to_MeanValue
plt.style.use('cyberpunk')
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 
#need to be set for the interactive GUI
%matplotlib ipympl

In [2]:
## test readout h5 file from recorder
filepath = "/masc_shared/ag_volz/Oliver_Maßmeyer/Perovskites/2024_07_19_2024_perovskite_NBD"
filename_save = 'NBD_08mrad_2c_4cm_001'

EELS_File= h5py.File(os.path.join(filepath,'NBD_08mrad_2c_4cm_001_data_000001.h5'),'r') 

In [3]:
EELS_MAP = EELS_File['entry/data/data']
EELS_MAP = hs.signals.Signal2D(EELS_MAP)
EELS_MAP.set_signal_type('EELS')

In [4]:
EELS_MAP.axes_manager

Navigation axis name,size,index,offset,scale,units
,17415,0,0.0,1.0,
Signal axis name,size,,offset,scale,units
,256,,0.0,1.0,
,256,,0.0,1.0,


In [5]:
EELS_MAP.plot()

In [6]:
## crop to correct size
x_scan_dim = 128 #gatan #cols
y_scan_dim = 128 #gatan #rows
expected_length = x_scan_dim*y_scan_dim

chosen_length = 17397
# chosen_length = x_scan_dim*y_scan_dim+1
offset = chosen_length-expected_length
additional_lines = np.floor(offset/x_scan_dim).astype(int)

start_value = 34
end_value = start_value+expected_length+additional_lines*x_scan_dim

#crop the dataset
EELS_MAP.crop(0, start=start_value, end=end_value)


In [7]:
additional_lines


7

In [7]:
## reshape and retile the dataset
test = EELS_MAP.data.reshape(y_scan_dim,x_scan_dim+additional_lines,256,256)
test = np.tile(test,(1,2,1,1)) #tiling

## convert array to EELS MAP to visualize the reshaping and tiling
EELS_MAP_reshaped =  hs.signals.Signal2D(test)
EELS_MAP_reshaped.set_signal_type('EELS')
EELS_MAP_reshaped.plot()

In [9]:
x_scan_dim+additional_lines

135

In [8]:
## induce row shift by indexing
slope = 0.49 ## adjust slope to match the row shift
roll_list = np.ceil(np.arange(1,x_scan_dim+1,1)/slope).astype(int)*-1 ## created shift list
print('Shift list', roll_list)

Shift list [  -3   -5   -7   -9  -11  -13  -15  -17  -19  -21  -23  -25  -27  -29
  -31  -33  -35  -37  -39  -41  -43  -45  -47  -49  -52  -54  -56  -58
  -60  -62  -64  -66  -68  -70  -72  -74  -76  -78  -80  -82  -84  -86
  -88  -90  -92  -94  -96  -98 -100 -103 -105 -107 -109 -111 -113 -115
 -117 -119 -121 -123 -125 -127 -129 -131 -133 -135 -137 -139 -141 -143
 -145 -147 -149 -152 -154 -156 -158 -160 -162 -164 -166 -168 -170 -172
 -174 -176 -178 -180 -182 -184 -186 -188 -190 -192 -194 -196 -198 -200
 -203 -205 -207 -209 -211 -213 -215 -217 -219 -221 -223 -225 -227 -229
 -231 -233 -235 -237 -239 -241 -243 -245 -247 -249 -252 -254 -256 -258
 -260 -262]


In [11]:
roll_list.shape

(128,)

In [9]:
## shift by indexing
rows, column_indices = np.ogrid[:test.shape[0], :test.shape[1]]
column_indices
rows
# Always use a negative shift, so that column_indices are valid.
# Alternative: r %= A.shape[1]
roll_list[roll_list < 0] += test.shape[1]
column_indices = column_indices - roll_list[:, np.newaxis]
# roll_list.shape
# column_indices.shape
# # rows.shape
result = test[rows, column_indices,:,:]
# result = test[rows, column_indices,:,:]

In [10]:
## convert array to EELS MAP and crop
EELS_MAP_reshaped =  hs.signals.Signal2D(result)
EELS_MAP_reshaped.set_signal_type('EELS')
## make final crop
EELS_MAP_reshaped.crop(0,start = 0, end = x_scan_dim)
EELS_MAP_reshaped.plot()

In [13]:
## store the converted dataset
EELS_MAP_reshaped.save(filename=os.path.join(filepath,filename_save), overwrite=True, extension='hspy')